In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import kagglehub

# Download latest version
path = kagglehub.dataset_download("akashdesarda/squad-v11")

print("Path to dataset files:", path)

/kaggle/input/squad-v11/SQuAD-v1.2.csv
/kaggle/input/squad-v11/SQuAD-v1.1.csv
Path to dataset files: /kaggle/input/squad-v11


In [7]:
import pandas as pd
import numpy as np 

df = pd.read_csv('/kaggle/input/squad-v11/SQuAD-v1.1.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87599 entries, 0 to 87598
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         87599 non-null  object
 1   context       87599 non-null  object
 2   question      87599 non-null  object
 3   answer        87596 non-null  object
 4   answer_start  87599 non-null  int64 
 5   answer_end    87599 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 4.0+ MB


In [8]:
df.dropna(subset = ['answer'],inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87596 entries, 0 to 87598
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         87596 non-null  object
 1   context       87596 non-null  object
 2   question      87596 non-null  object
 3   answer        87596 non-null  object
 4   answer_start  87596 non-null  int64 
 5   answer_end    87596 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 4.7+ MB


In [9]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [10]:
from transformers import AutoTokenizer 
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
def preprocessing(data):
  inputs = tokenizer(data['question'],data['context'],truncation=True,padding='max_length',max_length=384,return_offsets_mapping=True)
  start_positions = []
  end_positions = []

  for i,offset in enumerate(inputs['offset_mapping']):
    start = data['answer_start'][i]
    end = data['answer_end'][i]

    start_pos = end_pos = 0

    for j,mapping in enumerate(offset):
      if mapping[0]<= start and mapping[1] > start:
        start_pos = j
      if mapping[0] < end and mapping[1] >= end:
        end_pos = j

    start_positions.append(start_pos)
    end_positions.append(end_pos)

  inputs['start_positions'] = start_positions
  inputs['end_positions'] = end_positions
  inputs.pop("offset_mapping")

  return inputs

tokenized_dataset = dataset.map(preprocessing, batched=True)

Map:   0%|          | 0/87596 [00:00<?, ? examples/s]

In [12]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained('distilbert-base-uncased')

2025-08-21 21:51:00.336813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755813060.711311      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755813060.820810      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidi

In [18]:
from evaluate import load
import re

squad_metric = load("squad")
def normalize_answer(s):
    s = s.lower().strip()
    s = re.sub(r'\s+', ' ', s)
    s = re.sub(r'[^\w\s]', '', s)
    return s

In [20]:
import numpy as np
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction):
    start_logits, end_logits = p.predictions
    
    predictions = []
    references = []

    for i, ex in enumerate(dataset_splited['test']):
        start_idx = int(np.argmax(start_logits[i]))
        end_idx = int(np.argmax(end_logits[i]))
        tokens = tokenized_dataset[i]['input_ids'][start_idx:end_idx+1]
        answer_text = tokenizer.decode(tokens, skip_special_tokens=True)
        answer_text = normalize_answer(answer_text)
        if answer_text.strip() == "":
            answer_text = " "
        predictions.append({"id": str(i), "prediction_text": answer_text})

        ref_text = normalize_answer(ex['answer'])
        context_text = ex['context']
        start_pos = context_text.lower().find(ref_text)
        if start_pos == -1:
            start_pos = 0

        references.append({
            "id": str(i),
            "answers": {"text": [ref_text], "answer_start": [start_pos]}
        })
    
    results = squad_metric.compute(predictions=predictions, references=references)
    return {"exact_match": results["exact_match"], "f1": results["f1"]}


In [27]:
dataset_splited = tokenized_dataset.train_test_split(test_size=0.2)


from transformers import TrainingArguments, Trainer
from transformers import logging
logging.set_verbosity_info()

training_args = TrainingArguments(
    output_dir='./results',
    run_name=None,
    eval_strategy='epoch',
    logging_steps=100,
    fp16=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    dataloader_num_workers=4,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none",
    disable_tqdm=False
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = dataset_splited['train'],
    eval_dataset = dataset_splited['test'],
    compute_metrics=compute_metrics 
)
trainer.train()

PyTorch: setting up devices
Using auto half precision backend
The following columns in the Training set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: answer_start, answer_end, title, __index_level_0__, context, question, answer. If answer_start, answer_end, title, __index_level_0__, context, question, answer are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 70,076
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Training with DataParallel so batch size has been adjusted to: 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 10,950
  Number of trainable parameters = 66,364,418
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this war

Epoch,Training Loss,Validation Loss,Exact Match,F1
1,1.276400,1.202167,0.005708,0.557597
2,1.007200,1.113660,0.005708,0.594616
3,0.846500,1.111633,0.005708,0.603730
4,0.721100,1.141687,0.005708,0.599844
5,0.622200,1.200539,0.005708,0.598894


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/model.safetensors
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/model.safetensors
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/model.safetensors
/usr/local

TrainOutput(global_step=10950, training_loss=0.996869341009828, metrics={'train_runtime': 8322.089, 'train_samples_per_second': 42.102, 'train_steps_per_second': 1.316, 'total_flos': 3.433367489190912e+16, 'train_loss': 0.996869341009828, 'epoch': 5.0})

In [38]:
import re
import numpy as np
from transformers import pipeline
from evaluate import load
import torch

squad_metric = load("squad")

def normalize_answer_improved(s):
    s = s.lower()
    s = re.sub(r'\s+', ' ', s)
    s = re.sub(r'[^\w\s%.-]', '', s)
    return s.strip()

def evaluate_model_fast(model, tokenizer, dataset, split='test', batch_size=32, device='cuda'):
    model.eval()
    model.to(device)

    ds = dataset[split]
    input_ids = torch.tensor(ds['input_ids']).to(device)
    attention_mask = torch.tensor(ds['attention_mask']).to(device)
    contexts = ds['context']
    answers_text = ds['answer']

    all_start_preds = []
    all_end_preds = []

    for i in range(0, len(input_ids), batch_size):
        batch_input_ids = input_ids[i:i+batch_size]
        batch_attention_mask = attention_mask[i:i+batch_size]

        with torch.no_grad():
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits

            start_preds = torch.argmax(start_logits, dim=1).cpu().numpy()
            end_preds = torch.argmax(end_logits, dim=1).cpu().numpy()

            all_start_preds.extend(start_preds)
            all_end_preds.extend(end_preds)

    predictions = []
    references = []
    for i, (start_idx, end_idx) in enumerate(zip(all_start_preds, all_end_preds)):
        tokens = input_ids[i][start_idx:end_idx+1]
        pred_text = tokenizer.decode(tokens, skip_special_tokens=True)
        pred_text = normalize_answer(pred_text)

        ref_text = normalize_answer(answers_text[i])
        context_text = contexts[i]
        start_pos = context_text.lower().find(ref_text)
        if start_pos == -1:
            start_pos = 0

        predictions.append({"id": str(i), "prediction_text": pred_text})
        references.append({"id": str(i), "answers": {"text": [ref_text], "answer_start": [start_pos]}})

    results = squad_metric.compute(predictions=predictions, references=references)
    print(f"Exact Match: {results['exact_match']:.4f}")
    print(f"F1 Score: {results['f1']:.4f}")
    return results



In [39]:
results = evaluate_model_fast(model, tokenizer, dataset_splited, split='test', batch_size=32, device='cuda')

Exact Match: 55.6906
F1 Score: 70.3667


In [48]:
model.save_pretrained("/kaggle/working/my_model")
tokenizer.save_pretrained("/kaggle/working/my_model")

Configuration saved in /kaggle/working/my_model/config.json
Model weights saved in /kaggle/working/my_model/model.safetensors
tokenizer config file saved in /kaggle/working/my_model/tokenizer_config.json
Special tokens file saved in /kaggle/working/my_model/special_tokens_map.json


('/kaggle/working/my_model/tokenizer_config.json',
 '/kaggle/working/my_model/special_tokens_map.json',
 '/kaggle/working/my_model/vocab.txt',
 '/kaggle/working/my_model/added_tokens.json',
 '/kaggle/working/my_model/tokenizer.json')

In [52]:
ngrok.kill()

In [53]:
import os
from pyngrok import ngrok




app_code = """
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer
import streamlit as st

model = AutoModelForQuestionAnswering.from_pretrained("/kaggle/working/my_model")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/my_model")

QA_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)

st.title("📝 Question Answering App")

context = st.text_area("Context", height=200)
question = st.text_input("Question")

if st.button("Get Answer"):
    if context.strip() != "" and question.strip() != "":
        result = QA_pipeline(question=question, context=context)
        st.success(f"Answer: {result['answer']}\\n")
    else:
        st.warning("Please enter both context and question.")
"""

with open("/kaggle/working/app.py", "w") as f:
    f.write(app_code)

get_ipython().system_raw("streamlit run /kaggle/working/app.py &")

public_url = ngrok.connect(addr="8501", proto="http")
print("Your public URL:", public_url)


Your public URL: NgrokTunnel: "https://69a84deea57b.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8504
  Network URL: http://172.19.2.2:8504
  External URL: http://104.155.188.63:8504



Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
